# Building RAG Chatbots for Technical Documentation

## Table of contents

- [Introduction](#introduction)
- [Environment Setup](#environment-setup)
- [Load and split the document](#load-and-split-the-document)
- [Generate and store the embeddings](#generate-and-store-the-embeddings)

## Introduction 

This project involves implementing a retrieval augmented generation (RAG) with `LangChain` to create a chatbot for
answering questions about technical documentation. The document chosen for this assignment was the following: The European Union Medical Device Regulation - Regulation (EU) 2017/745 (EU MDR). 

## Environment Setup

Install the packages and dependencies to be used:

In [9]:
# install langchain
%pip install -qU langchain langchain-community langchain-chroma langchain-text-splitters unstructured sentence_transformers langchain-huggingface huggingface_hub pdfplumber

Note: you may need to restart the kernel to use updated packages.


## Load and split the document

In [2]:
# Using PDF document

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PDFPlumberLoader

loader = PDFPlumberLoader("document.pdf")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
pages = loader.load_and_split(text_splitter)

print(pages[0])


page_content='02017R0745 — EN — 09.07.2024 — 004.001 — 1
This text is meant purely as a documentation tool and has no legal effect. The Union's institutions do not assume any liability
for its contents. The authentic versions of the relevant acts, including their preambles, are those published in the Official
Journal of the European Union and available in EUR-Lex. Those official texts are directly accessible through the links
embedded in this document
►B REGULATION (EU) 2017/745 OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL
of 5 April 2017
on medical devices, amending Directive 2001/83/EC, Regulation (EC) No 178/2002 and
Regulation (EC) No 1223/2009 and repealing Council Directives 90/385/EEC and 93/42/EEC
(Text with EEA relevance)
(OJ L 117, 5.5.2017, p. 1)
Amended by:
Official Journal
No page date
►M1 Regulation (EU) 2020/561 of the European Parliament and of the L 130 18 24.4.2020
Council of 23 April 2020
►M2 Commission Delegated Regulation (EU) 2023/502 of 1 December 2022 L 70 1 8.

## Generate and store the embeddings

In [3]:
# Generate and store the embeddings
from langchain_chroma import Chroma
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

vectorstore = Chroma.from_documents(documents=pages, embedding=embeddings)


c:\Users\Athos\anaconda3\envs\TAAC_proj1\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
c:\Users\Athos\anaconda3\envs\TAAC_proj1\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


 ## Retrieve


In [34]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

retrieved_docs = retriever.invoke("Describe the use of harmonised standards")

for doc in retrieved_docs:
    print("page " + str(doc.metadata["page"] + 1) + ":", doc.page_content[:300])

page 16: management systems, risk management, post-market surveillance
systems, clinical investigations, clinical evaluation or post-market
clinical follow-up (‘PMCF’).
References in this Regulation to harmonised standards shall be
understood as meaning harmonised standards the references of which
have been 
page 168: personnel is informed of, any relevant standardisation activities and in
the activities of the notified body coordination group referred to in
Article 49 and that its assessment and decision-making personnel are
informed of all relevant legislation, guidance and best practice
documents adopted in th
page 147: (b) the method or methods used to demonstrate conformity with each
applicable general safety and performance requirement;
(c) the harmonised standards, CS or other solutions applied; and
(d) the precise identity of the controlled documents offering evidence of
conformity with each harmonised standar


## LLM

In [33]:
from langchain_huggingface.llms import HuggingFacePipeline
from transformers import pipeline, set_seed

generator = pipeline('text-generation', model='gpt2', max_length=1000, pad_token_id=50256)

llm = HuggingFacePipeline(pipeline=generator)

set_seed(42)
generator("Describe the use of harmonised standards", max_length=30, num_return_sequences=5)


c:\Users\Athos\anaconda3\envs\TAAC_proj1\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[{'generated_text': 'Describe the use of harmonised standards:\n\nThis is an extremely important question—how can a large proportion of people who may not speak the'},
 {'generated_text': 'Describe the use of harmonised standards to encourage good use of the same product\n\n(2)Where there is a strong preference among experts in'},
 {'generated_text': "Describe the use of harmonised standards, including the EU's 'Duke of Limbo' regulations as a 'guarantee that the countries"},
 {'generated_text': 'Describe the use of harmonised standards, one based on international practice of the EU.\n\n1. A national harmonised standard: whether under'},
 {'generated_text': 'Describe the use of harmonised standards. I find that most of the information I have for a particular type of document has nothing to do with the'}]

## Generate

In [36]:

from langchain_core.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible. Mention in which pages the answer is found.

{context}

Question: {question}

Helpful Answer:"""
prompt = PromptTemplate.from_template(template)


example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()
example_messages

print(example_messages[0].content)

Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible. Mention in which pages the answer is found.

filler context

Question: filler question

Helpful Answer:


In [37]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    formatted_docs = []
    for doc in docs:
        page_number = doc.metadata["page"] + 1 
        content_with_page = f"Page {page_number}:\n{doc.page_content}"
        formatted_docs.append(content_with_page)
    return "\n\n".join(formatted_docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print(rag_chain.invoke("Describe the use of harmonised standards"))

Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible. Mention in which pages the answer is found.

Page 16:
management systems, risk management, post-market surveillance
systems, clinical investigations, clinical evaluation or post-market
clinical follow-up (‘PMCF’).
References in this Regulation to harmonised standards shall be
understood as meaning harmonised standards the references of which
have been published in the Official Journal of the European Union.

Page 168:
personnel is informed of, any relevant standardisation activities and in
the activities of the notified body coordination group referred to in
Article 49 and that its assessment and decision-making personnel are
informed of all relevant legislation, guidance and best practice
documents adopted in the framework of this Regulation.
1.6.2. The 